In [1]:
import sys
import os
project_root = os.getcwd()  # This will use the current working directory
sys.path.append(os.path.join(project_root, 'Code'))
from utils import *
from data import *
from model import *
from train import *
from validate import *
from visualization import *

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


## Data

In [12]:
batch_size = 16
resize_size=(128,256)
preprocess_path = './Dataset_preprocess'
base_path = './Padel_data'
frame_info = 1
torch.manual_seed(42)
full_dataset = TrackNetDataset(base_path, resize_size=resize_size)
train_size = int(0.8 * len(full_dataset))
val_size = int(0.15 * len(full_dataset))
test_size = len(full_dataset) - train_size - val_size
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size, test_size])
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=0,
    pin_memory=True
)
val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=0,
    pin_memory=True
)
test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=0,
    pin_memory=True
)

## Train

In [13]:
epoch_num = 200
best_lev_dist = 1500
model_save_name = 'model_best_pandel.pth'
csv_save_path = 'training_metrics_pandel.csv'
use_eca =True
use_eca1=True

In [14]:
# Tracknet
gc.collect()
torch.cuda.empty_cache()
model = BallTrackerNet(use_eca=use_eca, use_eca1=use_eca1).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-2)
criterion = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.95, patience=1, verbose=True, threshold=1e-2)
train_loss_res = []
val_loss_res = []
val_dist_res =[]
precision_res = []
recall_res = []
f1_res = []

c:\Users\karby\anaconda3\envs\ml_env\lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [15]:

for epoch in range(0, epoch_num):
    print("\nEpoch: {}/{}".format(epoch+1, epoch_num))
    # Call train and validate 
    train_loss = train(model, train_loader, optimizer, criterion)
    train_loss_res.append(train_loss)
    val_loss, val_dist, precision, recall, f1 = validate(model, val_loader, criterion, min_dist=5)
    scheduler.step(val_dist)
    print("\nEpoch {}/{}: \t Train Loss {:.04f} ".format(
          epoch + 1,
          epoch_num,
          train_loss
          ))
    print("Val loss {:.04f} \t Val dist {:.04f} \t precision: {:.04f} \t recall: {:.04f}\t f1: {:.04f}".format(
          val_loss, val_dist, precision, recall, f1
          ))
    val_loss_res.append(val_loss)
    val_dist_res.append(val_dist.cpu().item())
    precision_res.append(precision)
    recall_res.append(recall)
    f1_res.append(f1)
    torch.cuda.empty_cache()
    if val_dist <= best_lev_dist:
      best_lev_dist = val_dist
      # Save your model checkpoint here
      print("Saving model")
      torch.save(model.state_dict(), model_save_name)
df_metrics = pd.DataFrame({
"train_loss": train_loss_res,
"val_loss": val_loss_res,
"val_dist": val_dist_res,
"precision": precision_res,
"recall": recall_res,
"f1": f1_res
})
df_metrics.to_csv('training_metrics.csv', index=False)


Epoch: 1/200



Epoch 1/200: 	 Train Loss 5.1985 
Val loss 7.8163 	 Val dist 80.2469 	 precision: 0.0000 	 recall: 0.0000	 f1: 0.0000
Saving model

Epoch: 2/200



Epoch 2/200: 	 Train Loss 4.4219 
Val loss 3.5999 	 Val dist 49.0439 	 precision: 0.0000 	 recall: 0.0000	 f1: 0.0000
Saving model

Epoch: 3/200



Epoch 3/200: 	 Train Loss 3.6875 
Val loss 4.5609 	 Val dist 58.0563 	 precision: 0.0000 	 recall: 0.0000	 f1: 0.0000

Epoch: 4/200



Epoch 4/200: 	 Train Loss 2.9739 
Val loss 2.5626 	 Val dist 91.2434 	 precision: 0.0000 	 recall: 0.0000	 f1: 0.0000

Epoch: 5/200


...


Epoch 196/200: 	 Train Loss 0.0363 
Val loss 0.0523 	 Val dist 1.4702 	 precision: 0.9730 	 recall: 1.0000	 f1: 0.9863

Epoch: 197/200



Epoch 197/200: 	 Train Loss 0.0351 
Val loss 0.0511 	 Val dist 1.2591 	 precision: 1.0000 	 recall: 1.0000	 f1: 1.0000
Saving model

Epoch: 198/200



Epoch 198/200: 	 Train Loss 0.0356 
Val loss 0.0527 	 Val dist 1.2599 	 precision: 0.9910 	 recall: 1.0000	 f1: 0.9955

Epoch: 199/200



Epoch 199/200: 	 Train Loss 0.0351 
Val loss 0.0504 	 Val dist 1.3725 	 precision: 1.0000 	 recall: 1.0000	 f1: 1.0000

Epoch: 200/200



Epoch 200/200: 	 Train Loss 0.0349 
Val loss 0.0503 	 Val dist 1.3148 	 precision: 1.0000 	 recall: 1.0000	 f1: 1.0000


## Test

In [16]:
model = BallTrackerNet(use_eca=use_eca, use_eca1=use_eca1).to(device)  # Ensure this matches your model
# Load the model state_dict
model.load_state_dict(torch.load(model_save_name))
model.to(device)
# Set model to evaluation mode
test_loss, test_dist, precision, recall, f1 = validate(model, test_loader, criterion, min_dist=2)
print("Test loss {:.04f} \t Test dist {:.04f} \t precision: {:.04f} \t recall: {:.04f}\t f1: {:.04f}".format(
          test_loss, test_dist, precision, recall, f1
          ))
torch.cuda.empty_cache()

C:\Users\karby\AppData\Local\Temp\ipykernel_10416\63641111.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_save_name))
           

Test loss 0.0420 	 Test dist 1.2411 	 precision: 0.8684 	 recall: 1.0000	 f1: 0.9296


## Visualization

In [17]:
visualize_predictions(model, test_loader, output_dir="visualizations_padel_ball", device=device)
print("Saved comparison frames to ./visualizations/")

Saved comparison frames to ./visualizations/
